In [2]:
!git clone https://github.com/ggml-org/llama.cpp

Cloning into 'llama.cpp'...
remote: Enumerating objects: 62652, done.
remote: Counting objects: 100% (168/168), done.
remote: Compressing objects: 100% (116/116), done.
remote: Total 62652 (delta 104), reused 57 (delta 52), pack-reused 62484 (from 4)
Receiving objects: 100% (62652/62652), 156.18 MiB | 31.16 MiB/s, done.
Resolving deltas: 100% (45416/45416), done.


In [3]:
!cd llama.cpp && cmake -B build -DGGML_CUDA=ON && cmake --build build --config Release

-- The C compiler identification is GNU 11.4.0
-- The CXX compiler identification is GNU 11.4.0
-- Detecting C compiler ABI info
-- Detecting C compiler ABI info - done
-- Check for working C compiler: /usr/bin/cc - skipped
-- Detecting C compile features
-- Detecting C compile features - done
-- Detecting CXX compiler ABI info
-- Detecting CXX compiler ABI info - done
-- Check for working CXX compiler: /usr/bin/c++ - skipped
-- Detecting CXX compile features
-- Detecting CXX compile features - done
CMAKE_BUILD_TYPE=Release
-- Found Git: /usr/bin/git (found version "2.34.1")
-- The ASM compiler identification is GNU
-- Found assembler: /usr/bin/cc
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD
-- Performing Test CMAKE_HAVE_LIBC_PTHREAD - Success
-- Found Threads: TRUE
-- Warning: ccache not found - consider installing it for faster compilation or disable this warning with GGML_CCACHE=OFF
-- CMAKE_SYSTEM_PROCESSOR: x86_64
-- GGML_SYSTEM_ARCH: x86
-- Including CPU backend
-- Found OpenMP_C: 

In [4]:
!pip install -r llama.cpp/requirements.txt

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/nightly, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/nightly, https://download.pytorch.org/whl/cpu, https://download.pytorch.org/whl/nightly
  Cloning https://github.com/huggingface/transformers (to revision v4.56.0-Embedding-Gemma-preview) to /tmp/pip-req-build-v88o6c8q
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-v88o6c8q
  Running command git checkout -q 60b68e304cf4b6569b0660a13b558b929d4b0e77
  Resolved https://github.com/huggingface/transformers to commit 60b68e304cf4b6569b0660a13b558b929d4b0e77
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
Ignoring torch: markers 'platform_machine == "s390x"' don't match your environment
Ignoring torch: markers 'platform_machine == "s390x"' don't

In [1]:
MODEL_ID = "mlabonne/EvolCodeLlama-7b"
MODEL_NAME = MODEL_ID.split("/")[-1]

In [10]:
!git lfs install
!git clone https://huggingface.co/{MODEL_ID}

Git LFS initialized.
Cloning into 'EvolCodeLlama-7b'...
remote: Enumerating objects: 40, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 40 (delta 0), reused 0 (delta 0), pack-reused 35 (from 1)
Unpacking objects: 100% (40/40), 486.18 KiB | 3.74 MiB/s, done.
Filtering content: 100% (7/7), 9.25 GiB | 4.09 MiB/s, done.
Encountered 2 file(s) that may not have been copied correctly on Windows:
	model-00001-of-00002.safetensors
	pytorch_model-00001-of-00002.bin

See: `git lfs help smudge` for more details.


In [11]:
fp16 = f"{MODEL_NAME}/{MODEL_NAME.lower()}.fp16.bin"
!python llama.cpp/examples/convert_legacy_llama.py {MODEL_NAME} --outtype f16 --outfile {fp16}

INFO:convert:Loading model file EvolCodeLlama-7b/model-00001-of-00002.safetensors
INFO:convert:Loading model file EvolCodeLlama-7b/model-00001-of-00002.safetensors
INFO:convert:Loading model file EvolCodeLlama-7b/model-00002-of-00002.safetensors
INFO:convert:params = Params(n_vocab=32016, n_embd=4096, n_layer=32, n_ctx=16384, n_ff=11008, n_head=32, n_head_kv=32, n_experts=None, n_experts_used=None, f_norm_eps=1e-05, rope_scaling_type=None, f_rope_freq_base=1000000, f_rope_scale=None, n_ctx_orig=None, rope_finetuned=None, ftype=<GGMLFileType.MostlyF16: 1>, path_model=PosixPath('EvolCodeLlama-7b'))
INFO:convert:Loaded vocab file PosixPath('EvolCodeLlama-7b/tokenizer.model'), type 'spm'
INFO:convert:model parameters count : (6738546688, 6738546688, 0) (6.7B)
INFO:convert:Vocab info: <SentencePieceVocab with 32016 base tokens and 0 added tokens>
INFO:convert:Special vocab info: <SpecialVocab with 0 merges, special tokens {'bos': 1, 'eos': 2, 'unk': 0}, add special tokens {'bos': False, 'se

In [21]:
!./llama.cpp/build/bin/llama-quantize

usage: ./llama.cpp/build/bin/llama-quantize [--help] [--allow-requantize] [--leave-output-tensor] [--pure] [--imatrix] [--include-weights]
       [--exclude-weights] [--output-tensor-type] [--token-embedding-type] [--tensor-type] [--prune-layers] [--keep-split] [--override-kv]
       model-f32.gguf [model-quant.gguf] type [nthreads]

  --allow-requantize: Allows requantizing tensors that have already been quantized. Warning: This can severely reduce quality compared to quantizing from 16bit or 32bit
  --leave-output-tensor: Will leave output.weight un(re)quantized. Increases model size but may also increase quality, especially when requantizing
  --pure: Disable k-quant mixtures and quantize all tensors to the same type
  --imatrix file_name: use data in file_name as importance matrix for quant optimizations
  --include-weights tensor_name: use importance matrix for this/these tensor(s)
  --exclude-weights tensor_name: use importance matrix for this/these tensor(s)
  --output-tensor-ty

In [34]:
METHODS = ["Q4_0", "Q4_1", "MXFP4_MOE", "Q5_0", "Q5_1", "IQ2_XXS", "IQ2_XS", "IQ2_S", "IQ2_M", "IQ1_S", "IQ1_M", "TQ1_0", "TQ2_0", "Q2_K", "Q2_K_S", "IQ3_XXS", "IQ3_S", "IQ3_M", "Q3_K", "IQ3_XS", "Q3_K_S", "Q3_K_M", "Q3_K_L", "IQ4_NL", "IQ4_XS", "Q4_K", "Q4_K_S", "Q4_K_M", "Q5_K", "Q5_K_S", "Q5_K_M", "Q6_K", "Q8_0", "F16", "BF16"]
METHODS[5]

'IQ2_XXS'

In [35]:
for METHOD in METHODS[5:]:
  qtype = f"{MODEL_NAME}/{MODEL_NAME.lower()}.{METHOD}.gguf"
  qtype = f"{MODEL_NAME}/{MODEL_NAME.lower()}.{METHOD}.gguf"
  !./llama.cpp/build/bin/llama-quantize {fp16} {qtype} {METHOD}

Streaming output truncated to the last 5000 lines.
[  43/ 291]                  blk.4.attn_q.weight - [ 4096,  4096,     1,     1], type =    f16, converting to q3_K .. size =    32.00 MiB ->     6.88 MiB
[  44/ 291]                  blk.4.attn_v.weight - [ 4096,  4096,     1,     1], type =    f16, converting to q3_K .. size =    32.00 MiB ->     6.88 MiB
[  45/ 291]                blk.4.ffn_down.weight - [11008,  4096,     1,     1], type =    f16, converting to q3_K .. size =    86.00 MiB ->    18.48 MiB
[  46/ 291]                blk.4.ffn_gate.weight - [ 4096, 11008,     1,     1], type =    f16, converting to q3_K .. size =    86.00 MiB ->    18.48 MiB
[  47/ 291]                blk.4.ffn_norm.weight - [ 4096,     1,     1,     1], type =    f32, size =    0.016 MiB
[  48/ 291]                  blk.4.ffn_up.weight - [ 4096, 11008,     1,     1], type =    f16, converting to q3_K .. size =    86.00 MiB ->    18.48 MiB
[  49/ 291]                  blk.5.attn_k.weight - [ 4096,  409

In [17]:
from huggingface_hub import create_repo, HfApi

In [ ]:
hf_token = ""
username = "Emanresu"

api = HfApi()

create_repo(
    repo_id=f"{username}/{MODEL_NAME}-GGUF",
    repo_type="model",
    exist_ok=True,
    token=hf_token
)

RepoUrl('https://huggingface.co/Emanresu/EvolCodeLlama-7b-GGUF', endpoint='https://huggingface.co', repo_type='model', repo_id='Emanresu/EvolCodeLlama-7b-GGUF')

In [37]:
api.upload_folder(
    folder_path=MODEL_NAME,
    repo_id=f"{username}/{MODEL_NAME}-GGUF",
    #allow_patterns=f"{MODEL_NAME.lower()}.{METHOD}.gguf",
    allow_patterns=f"*.gguf",
    token=hf_token
)

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

  ...lama-7b/evolcodellama-7b.TQ2_0.gguf:   1%|1         | 24.9MB / 1.85GB            

  ...lama-7b/evolcodellama-7b.IQ3_M.gguf:   0%|          | 8.23MB / 3.11GB            

  ...lama-7b/evolcodellama-7b.IQ3_S.gguf:   0%|          | 8.23MB / 2.95GB            

...

  ...lama-7b/evolcodellama-7b.IQ2_M.gguf:   0%|          |  655kB /  147MB            

  ...Llama-7b/evolcodellama-7b.Q4_1.gguf:   0%|          | 8.38MB / 4.24GB            

  ...Llama-7b/evolcodellama-7b.Q4_K.gguf:   0%|          | 8.38MB / 4.08GB            

  ...ama-7b/evolcodellama-7b.Q4_K_M.gguf:   0%|          | 8.38MB / 4.08GB            

CommitInfo(commit_url='https://huggingface.co/Emanresu/EvolCodeLlama-7b-GGUF/commit/ec1fb05ef2d2edca5977f695854eafe3a9e97e4c', commit_message='Upload folder using huggingface_hub', commit_description='', oid='ec1fb05ef2d2edca5977f695854eafe3a9e97e4c', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Emanresu/EvolCodeLlama-7b-GGUF', endpoint='https://huggingface.co', repo_type='model', repo_id='Emanresu/EvolCodeLlama-7b-GGUF'), pr_revision=None, pr_num=None)